In [9]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,AutoModelForSeq2SeqLM
import torch
import math


In [ ]:
df = pd.read_csv('done.csv')


In [6]:
print(torch.cuda.is_available())


True


In [5]:
# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

# Create pipeline
generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Extract only negative sentences
df_negative = df[df['label'] == 'Negative'].copy()

# Prepare list to store results
augmented_texts = []

# Batch size
batch_size = 100
num_batches = math.ceil(len(df_negative) / batch_size)

# Generate results in batches
for i in range(num_batches):
    batch = df_negative.iloc[i * batch_size : (i + 1) * batch_size]
    prompts = [f"Paraphrase this sentence with negative sentiment: {text}" for text in batch['reviews.text']]

    try:
        results = generator(prompts, max_new_tokens=50, batch_size=batch_size)
        for res in results:
            augmented_texts.append(res['generated_text'])
    except Exception as e:
        print(f"❌ Error in batch {i+1}: {e}")
        augmented_texts.extend([None] * len(prompts))  # Maintain order

# Combine results with original data
df_negative = df_negative.reset_index(drop=True)
df_negative['augmented_text'] = augmented_texts

# Show first 5 results
print(df_negative[['reviews.text', 'augmented_text']].head())

# Save to CSV file (optional)
df_negative.to_csv("augmented_negative_reviews_flan.csv", index=False)
print("✅ File saved: augmented_negative_reviews_flan.csv")

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (608 > 512). Running this sequence through the model will result in indexing errors


                                        reviews.text  \
0  Didn't have some of the features I was looking...   
1  i Bought this around black friday for $60 hopi...   
2  I bought this tablet for my 4 year old daughte...   
3  Didn't want to go to the sites amazon wanted m...   
4  Not a good product short battery life i can on...   

                                      augmented_text  
0  I'm not sure if it's a good thing or not, but ...  
1  i bought this tablet for $60... it failed so h...  
2  I bought this tablet for my 4 year old daughte...  
3                I don't want to go to amazon sites.  
4  Not a good product short battery life i can on...  
✅ تم حفظ الملف: augmented_negative_reviews_flan.csv


In [6]:
d = pd.read_csv('augmented_negative_reviews_flan.csv')
d.head()

,Unnamed: 0,reviews.text,label,augmented_text
0,117,Didn't have some of the features I was looking...,Negative,"I'm not sure if it's a good thing or not, but ..."
1,126,i Bought this around black friday for $60 hopi...,Negative,i bought this tablet for $60... it failed so h...
2,169,I bought this tablet for my 4 year old daughte...,Negative,I bought this tablet for my 4 year old daughte...
3,241,Didn't want to go to the sites amazon wanted m...,Negative,I don't want to go to amazon sites.
4,281,Not a good product short battery life i can on...,Negative,Not a good product short battery life i can on...


In [7]:
d.shape

(2440, 4)

# Neutral

In [10]:
# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

# Create pipeline
generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Extract only Neutral sentences
df_neutral = df[df['label'] == 'Neutral'].copy()

# Prepare list to store results
augmented_texts = []

# Batch size
batch_size = 100
num_batches = math.ceil(len(df_neutral) / batch_size)

# Generate results in batches
for i in range(num_batches):
    batch = df_neutral.iloc[i * batch_size : (i + 1) * batch_size]
    prompts = [f"Paraphrase this sentence with a neutral tone: {text}" for text in batch['reviews.text']]

    try:
        results = generator(prompts, max_new_tokens=50, batch_size=batch_size)
        for res in results:
            augmented_texts.append(res['generated_text'])
    except Exception as e:
        print(f"❌ Error in batch {i+1}: {e}")
        augmented_texts.extend([None] * len(prompts))  # Maintain order

# Combine results with original data
df_neutral = df_neutral.reset_index(drop=True)
df_neutral['augmented_text'] = augmented_texts

# Show first 5 results
print(df_neutral[['reviews.text', 'augmented_text']].head())

# Save to CSV file
df_neutral.to_csv("augmented_neutral_reviews_flan.csv", index=False)
print("✅ File saved: augmented_neutral_reviews_flan.csv")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors


                                        reviews.text  \
0  I was hoping to use Google launcher with this ...   
1  The tablet works fine. It is responsive with g...   
2  Fine for reading, ...haven't used it for anyth...   
3  Though I have got it for cheap price during bl...   
4  This is a good product for video, surfing and ...   

                                      augmented_text  
0  I was hoping to use Google launcher with this ...  
1  The tablet is good, but it is too tied to Amazon.  
2  I've used it for reading, but I've not used it...  
3  I have got it for cheap price during black fri...  
4  This version has dropped all android support a...  
✅ تم حفظ الملف: augmented_neutral_reviews_flan.csv


In [11]:
b = pd.read_csv('augmented_neutral_reviews_flan.csv')
b.head()

,Unnamed: 0,reviews.text,label,augmented_text
0,222,I was hoping to use Google launcher with this ...,Neutral,I was hoping to use Google launcher with this ...
1,226,The tablet works fine. It is responsive with g...,Neutral,"The tablet is good, but it is too tied to Amazon."
2,298,"Fine for reading, ...haven't used it for anyth...",Neutral,"I've used it for reading, but I've not used it..."
3,354,Though I have got it for cheap price during bl...,Neutral,I have got it for cheap price during black fri...
4,371,"This is a good product for video, surfing and ...",Neutral,This version has dropped all android support a...


In [15]:
b.augmented_text.iloc[100]

'I find it difficult to navigate. I also have an iPad mini.'

In [13]:
b.shape

(2790, 4)